In [ ]:
# %% [markdown]
# # Análisis de Niveles de Obesidad
# **Hipótesis:**
# 1. El consumo frecuente de alimentos calóricos (FAVC) está correlacionado con mayores niveles de obesidad.
# 2. La actividad física regular (FAF) reduce la probabilidad de obesidad.
# 3. El monitoreo de calorías (SCC) está asociado con menores niveles de sobrepeso.
# 4. El historial familiar de sobrepeso incrementa el riesgo de obesidad.

In [ ]:
# %%
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                            classification_report, mean_squared_error)
from imblearn.over_sampling import SMOTE

In [ ]:
# %% [markdown]
# ## Análisis Exploratorio de Datos (EDA)

In [ ]:
# %%
# Cargar datos
df = pd.read_csv("../data/ObesityDataSet.csv")

# Distribución de clases
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='NObeyesdad')
plt.title('Distribución de Niveles de Obesidad')
plt.show()

In [ ]:
# %%
# Relación entre variables
num_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
sns.pairplot(df[num_cols + ['NObeyesdad']], hue='NObeyesdad')
plt.show()

In [ ]:
# %%
# Codificación de variables categóricas
categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 
                   'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
df_encoded = pd.get_dummies(df, columns=categorical_cols)

# Balanceo de clases con SMOTE
X = df_encoded.drop('NObeyesdad', axis=1)
y = df_encoded['NObeyesdad']
X_res, y_res = SMOTE().fit_resample(X, y)

# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_res, test_size=0.2, random_state=42)

In [ ]:
# %% [markdown]
# ## Entrenamiento de Modelos

In [ ]:
# %%
# Configuración de modelos y parámetros
models = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga']}
    },
    'SVM': {
        'model': SVC(),
        'params': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    }
}

# Entrenamiento y evaluación
results = {}
for name, config in models.items():
    grid = GridSearchCV(config['model'], config['params'], cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)
    
    # Predicciones
    y_pred = grid.predict(X_test)
    
    # Métricas (solo las apropiadas para clasificación)
    results[name] = {
        'best_params': grid.best_params_,
        'accuracy': accuracy_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred)
    }

In [ ]:
# %% [markdown]
# ## Resultados y Evaluación

In [ ]:
# %%
# Mejor modelo
best_model = max(results, key=lambda x: results[x]['accuracy'])
print(f"Mejor modelo: {best_model}")

# Matriz de confusión
plt.figure(figsize=(10,7))
sns.heatmap(results[best_model]['confusion_matrix'], annot=True, fmt='d')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
# %%
# Reporte de métricas
print("Métricas clave:")
print(f"Precisión: {results[best_model]['accuracy']:.2f}")
print("\nReporte de clasificación completo:")
print(results[best_model]['classification_report'])